In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import re
import os

# 유튜브 크롤링 함수 생성

In [2]:
# 유튜브의 titles와 url을 반환하는 함수
def get_urls_from_youtube_with_keyword(keyword):
    titles = []
    urls = []
    
    search_keyword_encode = requests.utils.quote(keyword)
    
    url = "https://www.youtube.com/results?search_query=" + search_keyword_encode
    
    driver = wd.Chrome(executable_path="C:/Users/master14/Downloads/chromedriver_win32/chromedriver.exe") 
    
    driver.get(url)
    
    # 필터 버튼 클릭
    driver.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer/a').click()

    # 필터 기준 - 업로드 날짜 - 이번달
    upload_date = driver.find_element_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]')
    click_list_1 = upload_date.find_elements_by_id("endpoint")
    click_list_1[3].click()
    
    # 필터 창이 내려가는 속도에 맞추기 위해서 임의로 sleep한 후에 버튼 클릭
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="container"]/ytd-toggle-button-renderer/a').click()

    # 필터 기준 - 정렬 기준 - 조회수
    sorting_std = driver.find_element_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[5]')
    click_list_5 = sorting_std.find_elements_by_id("endpoint")
    click_list_5[1].click()
    time.sleep(2)

    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        time.sleep(3.0)
        
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        if new_page_height == last_page_height:
            break
            
        last_page_height = new_page_height
        
    html_source = driver.page_source
    
    driver.quit()
    
    soup = BeautifulSoup(html_source, 'lxml')
    
    datas = soup.select("a#video-title")

    for data in datas:
        title = data.text.replace('\n', '')
        url = "https://www.youtube.com/" + data.get('href')
        
        titles.append(title)
        urls.append(url)

    print(len(urls))
    return titles, urls

In [3]:
# urls 입력받아 html_sources를 반환하는 함수
def crawl_youtube_page_html_sources(urls):
    html_sources = []

    for i in range(len(urls)): #len(urls)
        driver = wd.Chrome(executable_path="C:/Users/master14/Downloads/chromedriver_win32/chromedriver.exe")
        
        driver.get(urls[i])

        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(3.0)
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

            if new_page_height == last_page_height:
                break
            last_page_height = new_page_height


        html_source = driver.page_source
        html_sources.append(html_source)
        print("OK")

        driver.quit()
    return html_sources

In [4]:
# 동영상 제목, 동영상 업로드일, 댓글 작성자, 댓글 내용, 댓글 작성일 크롤링하여 dataframe형식으로 반환하는 함수
def get_user_IDs_and_comments(html_sources):
    my_dataframes = []
    cnt = 0
    for html in html_sources:
        soup = BeautifulSoup(html, 'lxml')
        
        youtube_title = soup.select('#container > h1 > yt-formatted-string')
        
        youtube_title_date = soup.select('#date > yt-formatted-string')
        
        youtube_user_IDs = soup.select('#author-text > span')

        youtube_comments = soup.select('#content-text')
        
        youtube_date = soup.select('#header-author > yt-formatted-string > a')
                
        youtube_channel = soup.select('#text > a')
        
        youtube_like = soup.select('#text')
        
        youtube_dislike = soup.select('#text')
        
        youtube_comments_like = soup.select('#vote-count-middle')
        # print(youtube_comments_like)
        
        str_youtube_title = []
        str_youtube_title_date = []
        str_youtube_userIDs = []
        str_youtube_comments = []
        str_youtube_date = []
        str_youtube_channel = []
        str_youtube_like = []
        str_youtube_dislike = []
        str_youtube_comments_like = []
        
        cnt += 1 
        print(cnt)
        
        for i in range(len(youtube_user_IDs)):
            str_youtube_title.append(str(youtube_title[0].text))
            
            str_youtube_title_date.append(str(youtube_title_date[0].text))
            
            str_youtube_channel.append(str(youtube_channel[0].text))
            
            str_youtube_like.append(str(youtube_like[2].text))
            
            str_youtube_dislike.append(str(youtube_dislike[3].text))
            
            str_tmp = str(youtube_user_IDs[i].text)
            # print(str_tmp)
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('                ','')
            str_tmp = str_tmp.strip()
            str_youtube_userIDs.append(str_tmp)

            str_tmp = str(youtube_comments[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')
            str_youtube_comments.append(str_tmp)
            
            str_tmp = str(youtube_date[i].text) 
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('               ', '')
            str_youtube_date.append(str_tmp)
            
            str_tmp = str(youtube_comments_like[i].text)
            str_tmp = str_tmp.replace('\n', '')
            str_tmp = str_tmp.replace('\t', '')
            str_tmp = str_tmp.replace('  ', '')
            str_youtube_comments_like.append(str_tmp)
            # print(str_tmp)
        
        pd_data = {"Title" : str_youtube_title,
                   "Date" : str_youtube_title_date,
                   "ID" : str_youtube_userIDs, 
                   "Comment" : str_youtube_comments, 
                   "Comment_date" : str_youtube_date,
                   "Channel" : str_youtube_channel,
                   "Like" : str_youtube_like,
                   "Dislike" : str_youtube_dislike,
                   "Comment_like" : str_youtube_comments_like}
        

        youtube_pd = pd.DataFrame(pd_data)

        my_dataframes.append(youtube_pd)
        
    return my_dataframes

In [5]:
keyword = '2030 정치'

titles, url = get_urls_from_youtube_with_keyword(keyword)

html_sources = crawl_youtube_page_html_sources(url)

my_dataframes = get_user_IDs_and_comments(html_sources)

total = pd.DataFrame()
    
for i in range(len(my_dataframes)):
    total =  pd.concat([total,my_dataframes[i]])
    title = re.sub(' ','',keyword)
total.to_csv(f'data/{title}.csv', encoding='utf-8-sig')

9
OK
OK
OK
OK
OK
OK
OK
OK
OK
1
2
3
4
5
6
7
8
9
